<a href="https://colab.research.google.com/github/maxmatical/jax_projects/blob/master/02_jax_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random

defining hyperparams for NN

In [0]:
def random_layer_params(m, n, key, scale = 1e-2):
  # initialize the weights and bias for a hidden layer
  w, b = random.split(key)
  return scale*random.normal(w, (n,m)), scale*random.normal(b, (n,))

def init_nn_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

# definig parameters of the nn

szs = [28*28, 512, 256, 10] # flattened input will be 28*28
param_scale = 0.1
lr = 0.01
n_epochs = 10
bs = 128
n_classes = 10

params = init_nn_params(szs, random.PRNGKey(0))

defining NN

In [0]:
def relu(x):
  return np.maximum(0, x)

def net(params, input):
  # forward pass for a sinlge input
  out = input
  for w, b in params[:-1]: # go through hidden layers until classifier layer
    out = np.dot(w, out) + b
    out = relu(out)
  w_clas, b_clas = params[-1]
  logits = np.dot(w_clas, out)+b_clas
  # return logits
  return logits -logsumexp(logits)

checking to see if it works on a single sample

In [12]:
rand_input = random.normal(random.PRNGKey(1), (28*28,))
preds = net(params, rand_input)
print(preds.shape)

(10,)


using `vmap` to generate batch predictions

In [13]:
forward_batch = vmap(net, in_axes = (None, 0))
rand_input_batch = random.normal(random.PRNGKey(1), (128, 28*28,))
batch_preds = forward_batch(params, rand_input_batch)
print(batch_preds.shape)

(128, 10)


defining metrics and loss function

In [0]:
def one_hot(x, k, dtype = np.float32):
  # create 1-hot encoding of size k
  return np.array(x[:, None] == np.arrange(k), dtype)



def accuracy(params, input, targets):

def loss(params, input, targets):
  preds = forward_batch(params, input)
  return -np.mean(preds*targets)

def cross_entropy_loss(params, input, targets):
  # TODO
  # loss with just logits
